# 🎥 Face Movie Generator

Cette application recrée l'effet "Face Movie" de Picasa directement dans Google Colab.

## ✨ Fonctionnalités :
- Détection automatique des visages
- Recadrage centré
- Zoom et fondu entre les images
- Ajout optionnel de musique
- Téléchargement facile de la vidéo

💡 **Utilisation** :
1. Exécute toutes les cellules
2. Upload tes images et musique
3. Télécharge ta vidéo générée

In [ ]:
# 🛠️ Installer les bibliothèques nécessaires
!pip install gradio opencv-python moviepy numpy --quiet

In [ ]:
import os
import cv2
import numpy as np
import gradio as gr
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
from tempfile import TemporaryDirectory

# === Détection des visages ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_face(image, size=(640, 640)):
    img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    if len(faces) == 0:
        return None
    x, y, w, h = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)[0]
    margin = 0.4
    x1 = max(int(x - w*margin), 0)
    y1 = max(int(y - h*margin), 0)
    x2 = min(int(x + w + w*margin), img.shape[1])
    y2 = min(int(y + h + h*margin), img.shape[0])
    face_img = img[y1:y2, x1:x2]
    face_img = cv2.resize(face_img, size)
    return cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)

def create_face_movie(images, music=None):
    with TemporaryDirectory() as tempdir:
        clips = []
        for img in images:
            face = detect_face(img)
            if face is not None:
                clip = ImageClip(face).set_duration(2).resize(lambda t: 1 + 0.03 * t).crossfadein(1)
                clips.append(clip)
        if not clips:
            return "Aucun visage trouvé", None
        video = concatenate_videoclips(clips, method="compose")
        if music:
            video = video.set_audio(AudioFileClip(music.name).set_duration(video.duration))
        path = os.path.join(tempdir, "face_movie.mp4")
        video.write_videofile(path, fps=24, codec="libx264", audio_codec="aac")
        return "Vidéo prête 🎉", path

# Interface Gradio
iface = gr.Interface(
    fn=create_face_movie,
    inputs=[
        gr.File(label="Images", file_types=["image"], file_count="multiple"),
        gr.File(label="Musique (optionnel)", file_types=["audio"])
    ],
    outputs=[
        gr.Text(label="Statut"),
        gr.File(label="Télécharger la vidéo")
    ],
    title="Face Movie (Picasa-style)",
    description="Crée un diaporama vidéo centré sur les visages — compatible iPad"
)

iface.launch(share=True)